# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [1]:
import pyorient

In [2]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [3]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [4]:
client.db_create("gods",pyorient.DB_TYPE_GRAPH,pyorient.STORAGE_TYPE_PLOCAL)

**Q:** Connect your pyorient client to the `gods` database.

In [5]:
client.db_open("gods", "root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [6]:
import json
vertex = {'name': 'Zeus', 'symbol': 'thunder'}
json_string = json.dumps(vertex)

In [7]:
client.command(f"INSERT INTO V CONTENT {json_string}")

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [8]:
data = client.query("SELECT * FROM V")
for u in data:
    print(u)

{'@V':{'name': 'Zeus', 'symbol': 'thunder'},'version':1,'rid':'#9:0'}


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [9]:
vertex2 = {"name":"Héra", "symbol":"tiara"}
vertex3 = {"name":"Poséidon", "symbol":"trident"}
vertex4 = {"name":"Athena", "symbol":"helmet"}
vertex5 = {"name":"Arès", "symbol":"weapons"}

**Q:** Display all vertices with name = `Arès`

In [10]:
client.command(f"INSERT INTO V CONTENT {json.dumps(vertex2)}")
client.command(f"INSERT INTO V CONTENT {json.dumps(vertex3)}")
client.command(f"INSERT INTO V CONTENT {json.dumps(vertex4)}")
client.command(f"INSERT INTO V CONTENT {json.dumps(vertex5)}")

**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [11]:
edges = client.command("""
create edge
from (select from V where name = 'Zeus')
to (select from V where name = 'Poséidon')
""")

**Q:** Redisplay all vertices, discuss.

In [12]:
for val in client.command("SELECT * FROM V"):
    print(val)

{'@V':{'name': 'Zeus', 'symbol': 'thunder', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000020769543148>},'version':2,'rid':'#9:0'}
{'@V':{'name': 'Héra', 'symbol': 'tiara'},'version':1,'rid':'#10:0'}
{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000020769543448>},'version':2,'rid':'#11:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet'},'version':1,'rid':'#12:0'}
{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#13:0'}


**Q:** Display all edges. They are contained in the class `E`

In [13]:
for val in client.command("SELECT * FROM E"):
    print(val)

{'@E':{'out': <pyorient.otypes.OrientRecordLink object at 0x0000020769543948>, 'in': <pyorient.otypes.OrientRecordLink object at 0x0000020769543B08>},'version':1,'rid':'#17:0'}


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [14]:
# Fetch vertices using subqueries (assuming 'God' class exists)
zeus_vertex = client.query("SELECT FROM V WHERE name = 'Zeus'")[0]
poseidon_vertex = client.query("SELECT FROM V WHERE name = 'Poséidon'")[0]
Arès_vertex = client.query("SELECT FROM V WHERE name = 'Arès'")[0]
Athena_vertex = client.query("SELECT FROM V WHERE name = 'Athena'")[0]
Héra_vertex = client.query("SELECT FROM V WHERE name = 'Héra'")[0]

# Define the edges to be created
edges_to_create = [
    {"from": zeus_vertex._rid, "to": poseidon_vertex._rid, "content": {"kind": "sibling"}},   
    {"from": zeus_vertex._rid, "to": Arès_vertex._rid, "content": {"kind": "father"}},
    {"from": zeus_vertex._rid, "to": Athena_vertex._rid, "content": {"kind": "father"}},
    {"from": Héra_vertex._rid, "to": Arès_vertex._rid, "content": {"kind": "mother"}},
    {"from": Héra_vertex._rid, "to": zeus_vertex._rid, "content": {"kind": "sibling"}},
    {"from": poseidon_vertex._rid, "to": zeus_vertex._rid, "content": {"kind": "sibling"}},
]

# Create edges in a loop
for edge_data in edges_to_create:
    client.command(f"CREATE EDGE FROM {edge_data['from']} TO {edge_data['to']} CONTENT {edge_data['content']}")

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [15]:
for data in client.command("SELECT EXPAND(out()) FROM V where name ='Zeus'"):
    print(data)

{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002076954DDC8>, 'name': 'Poséidon', 'symbol': 'trident', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000020769554888>},'version':4,'rid':'#11:0'}
{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000020769554908>, 'name': 'Poséidon', 'symbol': 'trident', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000020769554A88>},'version':4,'rid':'#11:0'}
{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000020769554988>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000020769554E88>},'version':2,'rid':'#12:0'}


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [16]:
for data in client.command("SELECT EXPAND(in()) FROM V where name = 'Zeus'"):
    print(data)

{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000020769554188>, 'name': 'Héra', 'symbol': 'tiara'},'version':3,'rid':'#10:0'}
{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x0000020769554248>, 'name': 'Poséidon', 'symbol': 'trident', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000020769554588>},'version':4,'rid':'#11:0'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [17]:
for data in client.command("SELECT FROM V where out_ in (select FROM E where kind='mother')"):
    print(data)

{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000020769554508>, 'name': 'Héra', 'symbol': 'tiara'},'version':3,'rid':'#10:0'}


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [18]:
for data in client.command("SELECT expand(in()) FROM V where name='Zeus' and out_ in (select FROM E where kind = 'sibling')"):
    print(data.name)

Héra
Poséidon


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [19]:
client.db_create("eCommerce",pyorient.DB_TYPE_GRAPH,pyorient.STORAGE_TYPE_PLOCAL)
client.db_open("eCommerce", "root", ROOT_PASSWORD)

**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [20]:
client.command("create class Produit extends V")

[11]

In [21]:
product1 = {"name":"spaghetti",}
product2 = {"name":"bolognese sauce"}
product3 = {"name":"cheese"}
product4 = {"name":"apple"}
client.command(f"INSERT INTO Produit CONTENT {json.dumps(product1)}")
client.command(f"INSERT INTO Produit CONTENT {json.dumps(product2)}")
client.command(f"INSERT INTO Produit CONTENT {json.dumps(product3)}")
client.command(f"INSERT INTO Produit CONTENT {json.dumps(product4)}")

In [22]:
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)

{{'name': 'E'},'version':0,'rid':'#-2:0'}
{{'name': 'V'},'version':0,'rid':'#-2:1'}
{{'name': 'OUser'},'version':0,'rid':'#-2:2'}
{{'name': 'OIdentity'},'version':0,'rid':'#-2:3'}
{{'name': 'OFunction'},'version':0,'rid':'#-2:4'}
{{'name': 'ORole'},'version':0,'rid':'#-2:5'}
{{'name': 'OTriggered'},'version':0,'rid':'#-2:6'}
{{'name': 'OSequence'},'version':0,'rid':'#-2:7'}
{{'name': 'ORestricted'},'version':0,'rid':'#-2:8'}
{{'name': 'OSchedule'},'version':0,'rid':'#-2:9'}
{{'name': 'Produit'},'version':0,'rid':'#-2:10'}


**Q:** Create the following people: `peter`, `meredith`.

In [23]:
client.command("create class People extends V")
People1 = {"name":"peter",}
People2 = {"name":"meredith"}
client.command(f"INSERT INTO People CONTENT {json.dumps(People1)}")
client.command(f"INSERT INTO People CONTENT {json.dumps(People2)}")

In [24]:
for val in client.command("SELECT * FROM Produit"):
    print(val)

{'@Produit':{'name': 'spaghetti'},'version':1,'rid':'#25:0'}
{'@Produit':{'name': 'bolognese sauce'},'version':1,'rid':'#26:0'}
{'@Produit':{'name': 'cheese'},'version':1,'rid':'#27:0'}
{'@Produit':{'name': 'apple'},'version':1,'rid':'#28:0'}


**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [25]:
# Fetch vertices using subqueries (assuming 'God' class exists)
peter_vertex = client.query("SELECT FROM People WHERE name = 'peter'")[0]
meredith_vertex = client.query("SELECT FROM People WHERE name = 'meredith'")[0]
spaghetti_vertex = client.query("SELECT FROM Produit WHERE name = 'spaghetti'")[0]
cheese_vertex = client.query("SELECT FROM Produit WHERE name = 'cheese'")[0]
bolognese_sauce_vertex = client.query("SELECT FROM Produit WHERE name = 'bolognese sauce'")[0]
apple_vertex = client.query("SELECT FROM Produit WHERE name = 'apple'")[0]

# Define the edges to be created
edges_to_create = [
    {"from": peter_vertex._rid, "to": spaghetti_vertex._rid, "content": {"date": "20/01/2016"}},   
    {"from": peter_vertex._rid, "to": cheese_vertex._rid, "content": {"date": "20/01/2016"}},
    {"from": meredith_vertex._rid, "to": cheese_vertex._rid, "content": {"date": "22/01/2016"}},
    {"from": meredith_vertex._rid, "to": apple_vertex._rid, "content": {"date": "22/01/2016"}},
    {"from": meredith_vertex._rid, "to": bolognese_sauce_vertex._rid, "content": {"date": "22/01/2016"}},
    {"from": peter_vertex._rid, "to": spaghetti_vertex._rid, "content": {"date": "27/01/2016"}},
    {"from": peter_vertex._rid, "to": bolognese_sauce_vertex._rid, "content": {"date": "27/01/2016"}}
]

# Create edges in a loop
for edge_data in edges_to_create:
    client.command(f"CREATE EDGE FROM {edge_data['from']} TO {edge_data['to']} CONTENT {edge_data['content']}")

**Q:** Who bought Bolognese sauce?

In [26]:
for data in client.command("SELECT expand(in()) FROM Produit where name='bolognese sauce'" ):
    print(data.name)

meredith
peter


**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [27]:
data=client.command("select expand(in().out()) from Produit where name='bolognese sauce'")
datas=set([name.name for name in data])
for name in datas:
    print(name)

cheese
bolognese sauce
apple
spaghetti


In [28]:
for data in client.command("select date,in_,in(People),count(*) from E where count(*)>2 group by date,in_ ") :
    print(data)

{{'date': '27/01/2016', 'count': 2},'version':0,'rid':'#-2:2'}
{{'date': '22/01/2016', 'count': 3},'version':0,'rid':'#-2:1'}


## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.